# Project Galilei
### "Measure what is measurable, and make measurable what is not so."

## Attention Model Abstractive Text Summarization

    import

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import gensim
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, callbacks

from modules import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Attention, Dropout, Input, Embedding,Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2

    read

In [2]:
arxiv_meta = pd.read_json("../data/raw/arxiv-metadata-oai-snapshot.json", lines=True, nrows=10000)

In [3]:
arxiv_meta.head(5)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [4]:
arxiv_text = arxiv_meta[["id", "title", "abstract", "authors_parsed"]]

    preprocess

In [5]:
# grab set of stopwords from nltk and CV
sw = set(stopwords.words("english"))
cv = set(CountVectorizer(stop_words="english").get_stop_words())

# union of stopwrods from nltk and CV extended with custom stopwords
stops = list(set(cv | sw))
stops.extend(["php", "k", "ell", "n", "r", "t", "bf", "pi"])

In [6]:
arxiv_text["processed_abstract"] = preprocessing.free_form_preprocessor(arxiv_text, column_name="abstract", stops = stops)
arxiv_text["processed_title"] = preprocessing.free_form_preprocessor(arxiv_text, column_name="title", stops = ["php", "k", "ell", "n", "r", "t", "bf", "pi"])

C:\Users\mjwjl\AppData\Local\Temp/ipykernel_23732/1881454821.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arxiv_text["processed_abstract"] = preprocessing.free_form_preprocessor(arxiv_text, column_name="abstract", stops = stops)
C:\Users\mjwjl\AppData\Local\Temp/ipykernel_23732/1881454821.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arxiv_text["processed_title"] = preprocessing.free_form_preprocessor(arxiv_text, column_name="title", stops = ["php", "k", "ell", "n", "r", "t", "bf", "pi"])


In [7]:
# https://arxiv.org/pdf/1409.3215.pdf
# From what I understand it is debatable whether these are truly necessary. They're featured in many
# of the plate diagrams I reviewed so I include them here.
arxiv_text["processed_title"] = ["<SOS> " + x + " <EOS>" for x in arxiv_text["processed_title"]]

C:\Users\mjwjl\AppData\Local\Temp/ipykernel_23732/1995339452.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  arxiv_text["processed_title"] = ["<SOS> " + x + " <EOS>" for x in arxiv_text["processed_title"]]


### Tokenize Corpus

In [37]:
# abstract_tokens = [RegexpTokenizer(r"\w+").tokenize(x) for x in arxiv_text["processed_abstract"]]

In [44]:
abstract_tokens = [gensim.utils.simple_preprocess(x, deacc=True) for x in arxiv_text["processed_abstract"]]
title_tokens = [gensim.utils.simple_preprocess(x, deacc=True) for x in arxiv_text["processed_title"]]
# tokens = [gensim.utils.simple_preprocess(x, deacc=True) for x in arxiv_text["processed_title"]+arxiv_text["processed_abstract"]]

In [45]:
longest_abstract = max([len(abstract_tokens[i]) for i in range(len(abstract_tokens))])
longest_title = max([len(title_tokens[i]) for i in range(len(title_tokens))])
print(longest_abstract)
print(longest_title)

179
33


This post led me down a wierd path

In [29]:
# # This post led me down a wierd path
# # https://stackoverflow.com/questions/42064690/using-pre-trained-word2vec-with-lstm-for-word-generation
# X_train = [gensim.utils.simple_preprocess(x, deacc=True) for x in X_train]
# X_test = [gensim.utils.simple_preprocess(x, deacc=True) for x in X_test]
# y_train = [gensim.utils.simple_preprocess(x, deacc=True) for x in y_train]
# y_test = [gensim.utils.simple_preprocess(x, deacc=True) for x in y_test]

# X_train_zeroes = np.zeros([len(X_train),
#                            max([len(X_train[i]) for i in range(len(X_train))])])
# y_train_zeroes = np.zeros([len(y_train),
#                            max([len(y_train[i]) for i in range(len(y_train))])])

# for i, abstract in enumerate(X_train):
#     for j, token in enumerate(abstract):
#         X_train_zeroes = skip_embeddings.wv.key_to_index[token]

# #X, y = (arxiv_text["processed_abstract"], arxiv_text["processed_title"])

The right way

In [47]:
X_train, X_test, y_train, y_test = train_test_split(arxiv_text["processed_abstract"], arxiv_text["processed_title"], test_size=.2, random_state=137)

In [48]:
tokens = text.Tokenizer(num_words=10000)
tokens.fit_on_texts(pd.concat([X_train, X_test, y_train, y_test]))

X_train = tokens.texts_to_sequences(X_train)
X_test = tokens.texts_to_sequences(X_test)
y_train = tokens.texts_to_sequences(y_train)
y_test = tokens.texts_to_sequences(y_test)

In [50]:
dictionary = tokens.word_index

In [52]:
# Add padding and use pretrained Word2vec instead

### Embed Corpus

In [19]:
# The documentation for gensim is hit or miss. In this case the implementation of built in callbacks
# is obfuscated and possibly broken
# https://stackoverflow.com/questions/54888490/gensim-word2vec-print-log-loss

class callback(callbacks.CallbackAny2Vec):

    def __init__(self):
        self.epoch = 0
        self.loss_to_be_subbed = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subbed
        self.loss_to_be_subbed = loss
        print(f"Loss after epoch {self.epoch}: {loss_now}")
        self.epoch += 1

In [13]:
# skip_embeddings = Word2Vec(sentences=tokens, vector_size=128, alpha=0.02, window=15, min_count=5,
#                            workers=4, sg=1, max_vocab_size=25000, negative=5,
#                            epochs=100, compute_loss=True, callbacks=[callback()])

# skip_embeddings.save("../models/embeddings/skip_embedding_nlg.model")

In [14]:
# skip_embeddings = Word2Vec(sentences=tokens, vector_size=128, alpha=0.02, window=15,
#                            workers=4, sg=1, max_vocab_size=25000, negative=5,
#                            epochs=100, compute_loss=True, callbacks=[callback()])

# skip_embeddings.save("../models/embeddings/skip_embedding_nlg_no_min.model")

In [15]:
# cbow_embeddings = Word2Vec(sentences=tokens, vector_size=128, alpha=0.02, window=15,
#                            workers=4, sg=0, max_vocab_size=25000, negative=5,epochs=100,
#                            compute_loss=True, callbacks=[callback()])

# cbow_embeddings.save("../models/embeddings/cbow_embedding_nlg.model")

In [33]:
skip_embeddings = Word2Vec.load("../models/embeddings/skip_embedding_nlg.model")

skip_embeddings_no_min = Word2Vec.load("../models/embeddings/skip_embedding_nlg_no_min.model")

cbow_embedding = Word2Vec.load("../models/embeddings/cbow_embedding_nlg.model")

### Encoder-Decoder with Attention Model: Training

I am confident this will run through by the end of the week. Just need to homogenize/pad inputs and adjust input/encoder-decoder shapes accordingly.

In [53]:
# A Ten Minute Intro to seq2seq learning in keras
# https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

# Adapted from examples in keras documentation:
# https://keras.io/examples/generative/lstm_character_level_text_generation/
# https://keras.io/examples/nlp/text_generation_fnet/
# https://keras.io/examples/generative/text_generation_with_miniature_gpt/

# embeddings can be used as weights
# https://stackoverflow.com/questions/50311253/how-to-interpret-shape-of-word2vec-weights

encoder_input = Input(shape=(200, ))

embedding = Embedding(input_dim=,
                      output_dim=skip_embeddings.wv.vectors.shape[1],
                      input_length= 200, weights=[skip_embeddings.wv.vectors])(encoder_input)

encoder_layer_1 = LSTM(256, name="LSTM_1", return_state=True, recurrent_dropout=.1)#, return_sequences=True)
encoder_layer_1_output, e_h_1, e_c_1 = encoder_layer_1(embedding)
state_vector_1 = [e_h_1, e_c_1]

encoder_layer_2 = LSTM(256, name="LSTM_2", return_state=True, recurrent_dropout=.1)
encoder_layer_2_output, e_h_1, e_c_1 = encoder_layer_2(encoder_layer_1_output)
state_vector_2 = [e_h_2, e_c_2]

encoder_layer_3 = LSTM(256, name="LSTM_3", return_state=True, recurrent_dropout=.1)
encoder_layer_3_output, e_h_1, e_c_1 = encoder_layer_3(encoder_layer_2_output)
state_vector_3 = [e_h_3, e_c_3]

encoder_layer_4 = LSTM(256, name="LSTM_4", return_state=True, recurrent_dropout=.1)
encoder_layer_4_output, e_h_1, e_c_1 = encoder_layer_4(encoder_layer_3_output)
state_vector_4 = [e_h_4, e_c_4]

decoder_input = Input(shape=(200, ))

decoder = LSTM(256, name="decoder_LSTM", return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder(decoder_input, intial_state=state_vector_4)

# https://keras.io/api/layers/attention_layers/attention/
attention_head = Attention()
attention_dist, attention_value = attention_head([encoder_layer_4_output, decoder_outputs])

concat_output = Concatenate(axis=-1)
dec_att_out = concat_output([decoder_outputs, attention_dist])

y_hat = Dense(200, activation="softmax")(dec_att_out)

attention_model = Model([encoder_input, decoder_input], y_hat)

In [ ]:
# https://keras.io/api/losses/probabilistic_losses/#sparse_categorical_crossentropy-function
attention_model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy")

In [23]:
# tf.keras.utils.plot_model(encoder_decoder, show_shapes=True)

In [ ]:
attention_model.fit([encoder_input, decoder_input], decoder_target, batch_size=512, epochs=10,
                    validation_split=.2)

es = EarlyStopping(monitor="val_loss", min_delta=0, patience = 5, mode="auto")

history = attention_model.fit(X_train, y_train, validation_data=(X_test, y_test),
                       epochs=10, batch_size=100, callbacks=[es])

In [ ]:
plt.figure(figsize=(16, 9))

plt.plot(history.history["loss"], label="Training")
plt.plot(history.history["val_loss"] label="Test")
plt.legend()

### Inference Model